In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [0]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Stage One") \
    .getOrCreate()

In [0]:
music_data = 's3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz'

In [0]:
musics= spark.read.csv(music_data,header=True,sep='\t')

the total number of review

In [0]:
musics.count()#distinct

49

the number of unique user

In [0]:
musics.groupBy('customer_id').count().count()

48

the number of unique products

In [0]:
musics.groupBy('product_id').count().count()

49

the largest number of reviews published by a single user

In [0]:
musics.groupBy('customer_id'). count().agg({"count": "max"}).collect()

[Row(max(count)=2)]

the top 10 users ranked by the number of reviews they publish

In [0]:
from pyspark.sql.functions import *
musics.groupBy('customer_id').count().orderBy(desc("count")).show(10)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   47781982|    2|
|   52006292|    1|
|   20494593|    1|
|    1662075|    1|
|    6762003|    1|
|   43880421|    1|
|   43173394|    1|
|   13394189|    1|
|   32910511|    1|
|   41168357|    1|
+-----------+-----+
only showing top 10 rows



the median number of reviews published by a user

In [0]:
from pyspark.sql.window import Window
music0 = musics.groupBy('customer_id').count()
music1 = music0.withColumn("rowNum", row_number().over(Window.orderBy("count")))#partitionBy give different ranks for different XXX
rows = music1.agg(collect_list("rowNum")).collect()
begin = rows[0][0][0]
end = rows[0][0][-1]
med = int((begin + end) /2)
print (med)
music1.filter(music1.rowNum==med).show()

the largest number of reviews written for a single product

In [ ]:
musics.groupBy('product_id'). count().agg({"count": "max"}).collect()

the top 10 products ranked by the number of reviews they have

In [ ]:
musics.groupBy('product_id').count().orderBy(desc("count")).show(10)

the median number of reviews a product has

In [ ]:
music0 = musics.groupBy('product_id').count()
music1 = music0.withColumn("rowNum", row_number().over(Window.orderBy("count")))#partitionBy give different ranks for different XXX
rows = music1.agg(collect_list("rowNum")).collect()
begin = rows[0][0][0]
end = rows[0][0][-1]
med = int((begin + end) /2)
print (med)
music1.filter(music1.rowNum==med).show()